In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm import tqdm
from functools import lru_cache

In [3]:
bigrams = pd.read_csv('data/bigrams.txt', sep='\t', header=None, encoding='latin-1')

In [19]:
bigrams.head()

,0,1,2
0,275,a,a
1,31,a,aaa
2,29,a,all
3,45,a,an
4,192,a,and


### Create a dictionary of bigrams

In [20]:
bigram_counts = defaultdict(int, {(row[1], row[2]): int(row[0]) for row in bigrams.itertuples(index=False)})

In [29]:
# Convert to dictionary of word: count
vocabulary = defaultdict(int)
for (word1, word2), count in bigram_counts.items():
    vocabulary[word1] += count
    vocabulary[word2] += count
vocabulary_total = sum(vocabulary.values())

### Checker for known words

In [30]:
def known(words):
    """
    Method to get all known words from the input list of words
    :param words: list of str
    :return: set of str
    """
    return set(w for w in words if vocabulary[w] > 0)

### Get all words at a given Levenshtein distance

In [31]:
@lru_cache(maxsize=None)
def get_words_at_levenshtein_distance(word, distance):
    """
    Method to get all words at a given Levenshtein distance from the input word
    :param word: str
    :param distance: int
    :return: set of str
    """
    if distance == 0:
        return {word}

    letters = 'abcdefghijklmnopqrstuvwxyz'

    if distance == 1:
        parts = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        edits = (
            [a + b[1:] for a, b in parts if b] +  # Deletions
            [a + b[1] + b[0] + b[2:] for a, b in parts if len(b) > 1] +  # Transpositions
            [a + c + b[1:] for a, b in parts if b for c in letters] +  # Replacements
            [a + c + b for a, b in parts for c in letters]  # Insertions
        )
        return set(edits)

    candidates = set()
    for w in get_words_at_levenshtein_distance(word, distance - 1):
        candidates.update(get_words_at_levenshtein_distance(w, 1))

    return candidates

In [32]:
known(get_words_at_levenshtein_distance('keng', 1))

{'deng',
 'feng',
 'kang',
 'keg',
 'ken',
 'kent',
 'king',
 'kong',
 'kung',
 'peng',
 'seng'}

### Set of candidates for a given word

In [33]:
def find_candidates(word, max_distance=1):
    """
    Find candidate words within the given Levenshtein distance.

    :param word: str, the input word
    :param max_distance: int, maximum allowed Levenshtein distance (default is 1)
    :return: set of candidate words
    """
    possible_words = known([word])

    for dist in range(1, max_distance + 1):
        if possible_words:
            break
        possible_words.update(known(get_words_at_levenshtein_distance(word, dist)))

    return possible_words if possible_words else {word}

In [34]:
find_candidates('keng')

{'deng',
 'feng',
 'kang',
 'keg',
 'ken',
 'kent',
 'king',
 'kong',
 'kung',
 'peng',
 'seng'}

### Now we can get suggestions for a unigram and bigram

In [35]:
def unigram_suggestions(word, edit_distance):
    """
    Method to get suggestions for the input word
    :param word: 1 word
    :param edit_distance: int
    :return: suggestion
    """
    candidates = find_candidates(word, edit_distance)

    suggestions = []
    for word in candidates:
        prob = vocabulary[word] / vocabulary_total
        if prob > 0:
            suggestions.append((word, prob))

    return sorted(suggestions, key=lambda x: x[1], reverse=True)

In [36]:
def suggest_bigrams(bigram, distance=1, modify_left=True):
    """
    Generate bigram suggestions based on word candidates and their probabilities.

    :param bigram: tuple of two words (word1, word2)
    :param distance: int, maximum allowed Levenshtein distance
    :param modify_left: bool, whether to modify the left word or right word in the bigram
    :return: list of tuples with bigram suggestions and their probabilities
    """
    target_word = bigram[0] if modify_left else bigram[1]
    word_options = find_candidates(target_word, distance)

    candidate_bigrams = {(w, bigram[1]) if modify_left else (bigram[0], w) for w in word_options}

    suggestions = [
        (candidate, bigram_counts[candidate] / vocabulary[candidate[0]] if vocabulary[candidate[0]] > 0 else 0)
        for candidate in candidate_bigrams if bigram_counts.get(candidate, 0) > 0
    ]

    return sorted(suggestions, key=lambda x: x[1], reverse=True)

In [38]:
suggest_bigrams(('a', 'keng'), 1, modify_left=False)

[(('a', 'king'), 7.129329346254444e-05),
 (('a', 'keg'), 5.75227062167229e-06),
 (('a', 'ken'), 2.091734771517196e-06),
 (('a', 'kung'), 1.9174235405574297e-06),
 (('a', 'feng'), 1.6269048222911527e-06)]

### Autocorrect method

In [39]:
def correct_sentence(sentence, max_distance=1):
    """
    Corrects the input sentence by checking bigram and unigram suggestions.

    :param sentence: str, the sentence to autocorrect
    :param max_distance: int, maximum allowed Levenshtein distance (default is 1)
    :return: str, corrected sentence
    """
    words = sentence.split()

    if not words:
        return ""

    corrected_words = []

    for i, word in enumerate(words):
        # Get bigram suggestions for both directions (left and right)
        left_suggestions = suggest_bigrams((words[i], words[i + 1]), max_distance, modify_left=True) if i < len(words) - 1 else []
        left_suggestions = [(w, prob) for (w, _), prob in left_suggestions]

        right_suggestions = suggest_bigrams((words[i - 1], words[i]), max_distance, modify_left=False) if i > 0 else []
        right_suggestions = [(w, prob) for (_, w), prob in right_suggestions]

        # Combine all suggestions from bigrams
        suggestions = left_suggestions + right_suggestions

        # If no bigram suggestions, check for unigram suggestions
        if not suggestions and not known([word]):
            suggestions = unigram_suggestions(word, max_distance)

        # If still no suggestions, leave the word as is
        if not suggestions:
            corrected_words.append(word)
            continue

        # Sum the probabilities for each suggestion and pick the one with max probability
        word_probs = defaultdict(float)
        for suggestion, prob in suggestions:
            word_probs[suggestion] += prob

        # Choose the word with the highest probability
        corrected_words.append(max(word_probs, key=word_probs.get) if word_probs else word)

    return " ".join(corrected_words)

In [40]:
correct_sentence('Me wold like to go to the park but thera wus an acident on the way')

'he would like to go to the park but there was an accident on the way'

### Norvig's solution

In [41]:
def norvig_correction(word: str, edit_distance: int = 1) -> str:
    candidates = find_candidates(word, edit_distance)
    # choose the best candidate
    return max(candidates, key=vocabulary.get)


def norvig_autocorrect(sentence: str, edit_distance: int = 1) -> str:
    sentence_corrected = []

    for word in sentence.split():
        sentence_corrected.append(norvig_correction(word, edit_distance))

    return " ".join(sentence_corrected)

In [42]:
norvig_autocorrect('Me wold like to go to the park but thera wus an acident on the way')

'he would like to go to the park but there was an accident on the way'

## Evaluate on a test set

In [43]:
with open('data/TwelfthNight.txt', 'r') as file:
    test_set = file.readlines()

In [44]:
test_set[:10]

['\n',
 'Twelfth Night\n',
 '\n',
 'ACT I\n',
 "SCENE I. DUKE ORSINO's palace.\n",
 'Enter DUKE ORSINO, CURIO, and other Lords; Musicians attending\n',
 'DUKE ORSINO\n',
 'If music be the food of love, play on;\n',
 'Give me excess of it, that, surfeiting,\n',
 'The appetite may sicken, and so die.\n']

In [45]:
# add noise to words (change or delete some letters
def inject_noise(text, noise_probability):
    """
    Add random noise to the given text based on a specified probability.

    :param text: list of str, the lines of text to add noise to
    :param noise_probability: float, the probability of altering each word
    :return: list of str, the text with noise added
    """
    def alter_word(word):
        if np.random.rand() < noise_probability:
            idx = np.random.randint(len(word))
            replacement = np.random.choice([c for c in 'abcdefghijklmnopqrstuvwxyz' if c != word[idx]])
            return word[:idx] + replacement + word[idx + 1:]
        return word

    return [' '.join(alter_word(word) for word in line.split()) for line in text]

In [46]:
noised_text = inject_noise(test_set, 0.1)
noised_text[:10]

['',
 'Twelfth Night',
 '',
 'ACT I',
 "SCENE I. DUKE ORSINO's palace.",
 'Enter DUKE ORSINO, CURIO, and other Lords; Musicians attending',
 'DUKE ORSINO',
 'If music be the food of love, play on;',
 'Give me excess of it, that, surfeiting,',
 'The appetite may sicken, and so die.']

In [47]:
# evaluate the accuracy of the autocorrect method
def autocorrect_text(text, method):
    """
    Apply the specified autocorrect method to each line of the input text.

    :param text: list of str, the lines of text to correct
    :param method: function, the autocorrect method to use
    :return: list of str, the corrected text
    """
    return [method(line) for line in tqdm(text)]

def calculate_accuracy(corrected, original):
    """
    Evaluate the accuracy of the corrected text against the original text.

    :param corrected: list of str, the autocorrected text
    :param original: list of str, the original text
    :return: float, the accuracy as a ratio of correct words
    """
    total_correct = total_words = 0

    for corrected_line, original_line in zip(corrected, original):
        corrected_words = corrected_line.split() if isinstance(corrected_line, str) else corrected_line
        original_words = original_line.split()

        for c_word, o_word in zip(corrected_words, original_words):
            total_correct += c_word == o_word
            total_words += 1

    return total_correct / total_words if total_words > 0 else 0

corrected_text = autocorrect_text(noised_text, correct_sentence)
# change cirrected text to string
corrected_text = [line for line in corrected_text]

accuracy = calculate_accuracy(corrected_text, test_set)
accuracy

100%|██████████| 3697/3697 [00:03<00:00, 999.93it/s]


0.6816987329892069

In [48]:
def correct_norvig_text(text):
    """
    Apply Norvig's autocorrection to each line of the input text.

    :param text: list of str, the lines of text to correct
    :return: list of str, the corrected text
    """
    return [norvig_autocorrect(line) for line in tqdm(text)]

norvig_corrected_text = correct_norvig_text(noised_text)
# change corrected text to string
norvig_corrected_text = [line for line in norvig_corrected_text]

norvig_accuracy = calculate_accuracy(norvig_corrected_text, test_set)
norvig_accuracy

100%|██████████| 3697/3697 [00:01<00:00, 3349.59it/s]


0.6809948381041765

# Conclusion
The difference is obsolete between the two methods, as both have different accuracy .681 and .680.

I think the difference is not much, because of Shakespeare's text of Twelfth Night, which is quite old and has a lot of words that are not used today. This makes it hard for both methods to correct the text, as they are not trained on such old text (so bigrams).

Still results of my solution are better due to a more complex approach I propose.